In [1]:
import sys # aby importy z dołu działały
sys.path.append(r"D:\MyProjects_4Fun\projects\World of Warcraft\python-etl")

In [2]:
from scraper_wiki_main import parsuj_misje_z_url, dekompresuj_html, pobierz_soup
import json
import copy
from bs4 import BeautifulSoup
import zlib
import base64
from sqlalchemy import text

from moduly.db_core import utworz_engine_do_db

In [3]:
s = pobierz_soup(url="https://www.wowhead.com/quest=59350/a-druids-form")

In [ ]:
kontener = s.find(class_="page-controls")


<div class="page-controls"></div>

In [21]:
kontener.parent.get_text()

"\n \n\n\n\n\n\n\n\n\n\nQuick Facts \n\n\n\n\n\n\n\n\n\n\n\nStorylines \n\n\n\n\nA Dark Ritual\n\nRide of the Scientifically Enhanced BoarRepair YourselfStocking Up on SuppliesThe Harpy ProblemThe Rescue of Meredy HuntswellA Druid's FormWho Lurks in the PitHarpy CullingPurge the TotemsMessage to BaseWestward BoundTo Darkmaul CitadelRight Beneath Their EyesLike Ogres to the SlaughterCatapult DestructionControlling their StonesDungeon: Darkmaul CitadelAn End to Beginnings\n\n\n\nThe Missing Expedition\n\nWarming UpStand Your GroundBrace for ImpactMurloc ManiaEmergency First AidFinding the Lost ExpeditionCooking MeatEnhanced Combat TacticsNorthboundDown with the QuilboarQuilboar Shadow MagicThe Scout-o-Matic 5000Re-sizing the SituationRide of the Scientifically Enhanced BoarThe Harpy ProblemWestward BoundStocking Up on SuppliesRepair YourselfTo Darkmaul CitadelRight Beneath Their EyesLike Ogres to the SlaughterCatapult DestructionControlling their StonesDungeon: Darkmaul CitadelWho Lurks 

In [3]:
url = "https://warcraft.wiki.gg/wiki/Enhanced_Combat_Tactics#Alliance"

In [4]:
# wynik = parsuj_misje_z_url(url)

# kopia_do_druku = copy.deepcopy(wynik)
# kopia_do_druku["Źródło"]["html_skompresowany"] = "--- [UKRYTO DLA CZYTELNOŚCI] ---"

# print(json.dumps(kopia_do_druku, indent=4, ensure_ascii=False))

In [5]:
silnik = utworz_engine_do_db()

In [ ]:
with silnik.connect() as conn:
    conn.execute(text("""
    SELECT DISTINCT MISJA_URL_WOWHEAD
    FROM dbo.MISJE
"""))

In [5]:
soup = pobierz_soup(url)
x = soup.select_one("#bodyContent")

In [10]:
def wyciagnij_wowhead_data(silnik) -> tuple[int | None, str | None]:
    misja_id = 1
    q_select_misja = text("""
        SELECT TOP 1 HTML_SKOMPRESOWANY
        FROM dbo.ZRODLO
        WHERE MISJA_ID_MOJE_FK = :misja_id
        ORDER BY DATA_WYSCRAPOWANIA DESC
    """)

    with silnik.connect() as conn:
        x = conn.execute(q_select_misja, {"misja_id": misja_id}).scalar_one()
    skompresowane_bajty = base64.b64decode(x)
    tekst_html = zlib.decompress(skompresowane_bajty).decode("utf-8")

    linki = soup.select("a[href*='wowhead.com']")
    for link in linki:
        href = link.get("href", "")
        if "/quest=" in href or "quest=" in href:
            try:
                raw_id = href.split("quest=")[-1].split("&")[0]
                raw_id = raw_id.split("#")[0]
                return int(raw_id), href
            except ValueError:
                continue
    return None, None

In [7]:
wyciagnij_wowhead_data(soup)

(59254, 'https://www.wowhead.com/quest=59254')